<a href="https://colab.research.google.com/github/MehrdadJalali-AI/GraphMOF-AI/blob/main/Neo4j_MOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install neo4j


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 6.9 MB/s eta 0:00:00


In [1]:
from neo4j import GraphDatabase

# Neo4j connection details
URI = "bolt://localhost:7474"  # Change if your Neo4j is on another server
USERNAME = "neo4j"
PASSWORD = "12345678"  # Change to your actual password

class KnowledgeGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_nodes_and_relationships(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")  # Clear existing graph

            # Creating sample nodes
            session.run("CREATE (a:Person {name: 'Alice'})")
            session.run("CREATE (b:Person {name: 'Bob'})")
            session.run("CREATE (c:Company {name: 'TechCorp'})")

            # Creating relationships
            session.run("""
                MATCH (a:Person {name: 'Alice'}), (b:Person {name: 'Bob'})
                CREATE (a)-[:KNOWS]->(b)
            """)
            session.run("""
                MATCH (a:Person {name: 'Alice'}), (c:Company {name: 'TechCorp'})
                CREATE (a)-[:WORKS_FOR]->(c)
            """)

    def query_graph(self):
        with self.driver.session() as session:
            result = session.run("MATCH (a)-[r]->(b) RETURN a.name, type(r), b.name")
            for record in result:
                print(f"{record['a.name']} -[{record['type(r)']}]-> {record['b.name']}")

# Run the script
if __name__ == "__main__":
    kg = KnowledgeGraph(URI, USERNAME, PASSWORD)
    kg.create_nodes_and_relationships()
    print("Knowledge Graph Created:")
    kg.query_graph()
    kg.close()


ModuleNotFoundError: No module named 'neo4j'

In [ ]:
from py2neo import Graph, Node, Relationship
import pandas as pd

# Connect to Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

# Load MOF dataset
file_path = "MOF.csv"
mof_data = pd.read_csv(file_path)

# Clear existing data
graph.delete_all()

# Create MOF nodes and relationships
for _, row in mof_data.iterrows():
    mof_node = Node("MOF", name=row["Refcode"])

    # Metal Ion Node
    if pd.notna(row["metals"]):
        metal_node = Node("Metal", name=row["metals"])
        graph.merge(metal_node, "Metal", "name")
        graph.create(Relationship(mof_node, "HAS_METAL", metal_node))

    # Organic Linker Node
    if pd.notna(row["SMILES"]):
        linker_node = Node("Linker", smiles=row["SMILES"])
        graph.merge(linker_node, "Linker", "smiles")
        graph.create(Relationship(mof_node, "HAS_LINKER", linker_node))

    # Structural Properties
    if pd.notna(row["pld (A)"]):
        pld_node = Node("Property", name="PLD", value=float(row["pld (A)"]))
        graph.create(pld_node)
        graph.create(Relationship(mof_node, "HAS_PROPERTY", pld_node))

    if pd.notna(row["asa (A^2)"]):
        sa_node = Node("Property", name="Surface Area", value=float(row["asa (A^2)"]))
        graph.create(sa_node)
        graph.create(Relationship(mof_node, "HAS_PROPERTY", sa_node))

    if pd.notna(row["void_fraction"]):
        vf_node = Node("Property", name="Void Fraction", value=float(row["void_fraction"]))
        graph.create(vf_node)
        graph.create(Relationship(mof_node, "HAS_PROPERTY", vf_node))

    graph.create(mof_node)

print("MOF data successfully stored in Neo4j.")


In [ ]:
neo4j status


SyntaxError: invalid syntax (<ipython-input-3-9ac429d5d06b>, line 1)